<a href="https://colab.research.google.com/github/BhojRajBist/BhojRajBist/blob/main/Async_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install asyncio

In [1]:
async def my_function():
    await some_other_function()


In [4]:
import asyncio

async def greet():
    print("Hi, waiting for 2 seconds...")
    await asyncio.sleep(2)
    print("Done waiting!")

# Run the async function
await greet()


Hi, waiting for 2 seconds...
Done waiting!


In [5]:
async def task(name, delay):
    print(f"{name} started...")
    await asyncio.sleep(delay)
    print(f"{name} finished after {delay} seconds")

# Run them together
await asyncio.gather(
    task("Task 1", 2),
    task("Task 2", 3),
    task("Task 3", 1),
)


Task 1 started...
Task 2 started...
Task 3 started...
Task 3 finished after 1 seconds
Task 1 finished after 2 seconds
Task 2 finished after 3 seconds


[None, None, None]